In [1]:
from transformers import RobertaForMaskedLM, RobertaTokenizer
import torch
from tqdm import tqdm

In [37]:
model = RobertaForMaskedLM.from_pretrained('./output/')
tokenizer = RobertaTokenizer.from_pretrained('./output/')

12/01/2019 19:52:33 - INFO - transformers.configuration_utils -   loading configuration file ./output/config.json
12/01/2019 19:52:33 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 1,
  "use_bfloat16": false,
  "vocab_size": 50265
}

12/01/2019 19:52:33 - INFO - transformers.modeling_utils -   loading weights file ./output/pytorch_model.bin
12/01/2019 19:52:37 - INFO - transformers.tokenization_utils -   Model name './output/' not found in model shortcut name list (robe

In [38]:
import os
import pandas as pd
from random import choice

import nltk

In [39]:
path_to_intents = os.path.join('data', 'raw')
intents = os.listdir(path_to_intents)
get_path = lambda x: os.path.join('data', 'raw', x, x + ".csv")

In [40]:
os.chdir('helpers')
from analyze import questions, entities, get_data
data = get_data()
os.chdir('..')

In [69]:
from collections import defaultdict
text = defaultdict(list)
encoded_text = defaultdict(list)

ans = defaultdict(list)
response = defaultdict(list)

scores = defaultdict(list)

for (_questions, intent, entity) in zip(questions, intents, entities):
    question = choice(_questions)
    
    for row in data[intent]['df']['text']:
        text[intent].append(row.strip() + '. ' + question.strip())
        
    for row in data[intent]['df'][entity]:
        ans[intent].append((str(row)).split())

In [70]:
for intent in intents:
    for row in text[intent]:
        encoded_text[intent].append(torch.tensor([tokenizer.encode(row, add_special_tokens=True)]))

In [71]:
encoded_text['BookRestaurant'][0]

tensor([[   0, 6298,   20, 2367,  953, 1437, 2391,   11, 2808,   13, 5996,    4,
         6834,  317,  116,    2]])

In [72]:
text['BookRestaurant'][0]

'book The Middle East  restaurant in IN for noon. Which place?'

In [73]:
ans['BookRestaurant'][0]

['The', 'Middle', 'East', 'in', 'IN']

In [74]:
with torch.no_grad():
    for intent in intents:
        for i, row in tqdm(enumerate(encoded_text[intent])):
            out = model(row)
            actual_out = model((torch.argmax(out[0][0], dim=1)).view(1, -1))
            print(actual_out)
            response[intent].append(tokenizer.decode(torch.argmax(out[0][0], dim=1).tolist()).split())
            print(response[intent][-1])
            break
            score = nltk.translate.bleu_score.sentence_bleu(
                    [ans[intent][i]],
                    response[intent][i],
                    smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method4,
                    auto_reweigh=True
                )
            

0it [00:00, ?it/s]
0it [00:00, ?it/s]

(tensor([[[ 9.9406, -2.9603,  8.3970,  ...,  1.5137,  3.6402,  6.3281],
         [ 4.4129, -2.6280, 10.0783,  ...,  0.1638,  3.3117,  5.1125],
         [ 5.7099, -2.0169,  3.5284,  ...,  1.9832,  2.4438,  3.9433],
         ...,
         [ 7.8212, -2.1413,  4.1847,  ...,  2.2004,  3.1155,  5.6780],
         [ 6.4131, -2.7355,  3.2601,  ...,  2.8562,  3.9446,  7.4909],
         [10.9007, -2.2821,  9.5213,  ...,  5.3673,  4.9585,  9.6004]]]),)
['<s>Add', 'another', 'song', 'to', 'the', 'Cita', 'RomÃ¡ntica', 'playlist.', 'What', 'should', 'I', 'add', 'to?']
(tensor([[[15.7489, -2.8303,  8.5910,  ...,  1.7682,  3.9350,  8.7439],
         [-0.3903, -4.4944,  6.2178,  ..., -4.6338, -3.0864,  1.7993],
         [ 5.2035, -3.0675,  6.2530,  ...,  0.7793,  3.5323,  3.9874],
         ...,
         [-3.0049, -3.4744,  3.5694,  ..., -1.9881,  1.2798, -2.0960],
         [ 5.4573, -2.9659,  3.2425,  ...,  1.2363,  2.7138,  5.9312],
         [ 9.6744, -2.5804,  9.8234,  ...,  4.0117,  4.8970,  9.3715]]

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

(tensor([[[ 9.5343, -2.8868,  6.4878,  ...,  1.7897,  3.1463,  6.7134],
         [ 4.7981, -3.0404, 12.9067,  ...,  1.1233,  3.1445,  7.0639],
         [ 2.2193, -3.1450,  3.9553,  ...,  2.3891,  2.1724,  5.7924],
         ...,
         [-0.4668, -3.3859,  3.1795,  ...,  0.3377,  0.5385,  3.1542],
         [ 2.5330, -3.0407,  5.0515,  ...,  2.9549,  1.1939,  5.4320],
         [ 3.7759, -3.5574,  9.0519,  ..., -2.4075, -2.4362,  6.5678]]]),)
['<s>What', 'will', 'the', 'weather', 'be', 'this', 'year', 'in', 'Horseshoe', 'Lake', 'State', 'Fish', 'and', 'Wildlife', 'Area?', 'I', 'will', 'tell', 'you', 'the', 'weather', 'for']
(tensor([[[21.7438, -2.9834,  8.9592,  ...,  2.2917,  4.5047, 10.9875],
         [ 6.9856, -3.0568, 11.7913,  ...,  1.4867,  4.0744,  7.7476],
         [ 0.6561, -3.3759,  3.9537,  ..., -0.9491,  1.2122,  3.5243],
         ...,
         [-1.5928, -2.7800,  3.1824,  ...,  1.6826,  1.0537,  2.7808],
         [ 0.3491, -3.9352,  4.5772,  ...,  0.9896,  1.3818,  2.2575],


0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

['<s>I', 'need', 'to', 'hear', 'the', 'song', 'Aspro', 'Mavro', 'from', 'Bill', 'Szymczyk', 'on', 'Youtube', 'I', 'will', 'play</s>']
(tensor([[[ 9.2953, -3.1320,  7.4972,  ...,  1.9521,  2.8192,  6.8728],
         [ 3.0743, -4.4380,  6.8570,  ..., -2.9806, -3.6016,  0.8754],
         [ 3.8165, -2.6787,  8.5445,  ...,  1.3608,  2.0882,  3.8909],
         ...,
         [-3.4266, -4.2963,  5.7800,  ...,  0.9868,  1.1777,  1.9817],
         [ 4.0788, -3.2559,  6.2211,  ...,  1.1942, -1.5063,  6.8653],
         [ 8.7478, -3.5071, 13.5857,  ...,  3.2865,  0.7605,  7.4194]]]),)
['<s>rate', 'The', 'Lotus', 'and', 'the', 'Storm', 'zero', 'of', '6', 'I', 'will', 'rate', 'it']


0it [00:00, ?it/s]
0it [00:00, ?it/s]

(tensor([[[12.2224, -2.8887,  7.5436,  ...,  1.6901,  3.7063,  7.6917],
         [ 3.5270, -3.9497,  4.6634,  ..., -2.2576, -1.8791,  4.0627],
         [ 0.9311, -2.9260,  2.4014,  ...,  1.0564,  2.7370,  7.1741],
         ...,
         [ 2.8491, -3.0966,  5.5013,  ...,  4.2222,  2.6626,  5.2596],
         [ 8.1605, -2.8175,  4.9231,  ...,  3.2799,  4.1806,  7.8245],
         [10.1431, -2.4684, 10.9726,  ...,  5.1504,  3.8828, 10.6926]]]),)
['<s>find', 'the', 'soundtrack', 'titled', 'This', 'Side', 'of', 'Paradise', 'What', 'should', 'I', 'look', 'for?']


0it [00:00, ?it/s]

(tensor([[[11.8776, -3.5336,  7.4991,  ...,  0.0739,  1.7538,  7.2090],
         [ 4.6219, -3.1463, 13.0176,  ...,  0.8489,  3.4118,  6.5938],
         [ 1.8810, -3.6164,  7.3875,  ...,  1.6342, -0.7474,  2.8845],
         ...,
         [-4.1597, -4.0840,  2.6684,  ...,  1.8363, -0.2410,  2.0421],
         [ 2.4781, -3.7044,  6.4244,  ...,  3.7775,  0.6670,  6.9013],
         [ 6.1845, -3.9607,  9.8287,  ..., -2.5426, -3.8224,  4.9154]]]),)
['<s>What', 'are', 'the', 'movie', 'schedule', 'at', 'Malco', 'Theatres', 'I', 'will', 'look', 'for']


In [58]:
" ".join(response['BookRestaurant'][0])

'<s>book The Middle East restaurant in IN for noon Where do they want to eat?'

In [59]:
for intent in intents:
    print(f"For {intent}, avg. BLEU score is {sum(scores[intent]) / len(scores[intent])}")

ZeroDivisionError: division by zero

In [75]:
os.chdir('transformers')
from examples.run_generation import sample_sequence
os.chdir('..')
_out = sample_sequence(model, 30, encoded_text['AddToPlaylist'][1][0], top_p = 0.9)

100%|█████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:03<00:00,  7.68it/s]


In [76]:
encoded_text['BookRestaurant'][0][0]

tensor([   0, 6298,   20, 2367,  953, 1437, 2391,   11, 2808,   13, 5996,    4,
        6834,  317,  116,    2])

In [77]:
tokenizer

In [78]:
tokenizer.decode(_out.squeeze(0).tolist())

'<s>add clem burke in my playlist Pre-Party R&B Jams. What should I add to?</s>\nmymyJPPRRRRRRRRRRRRRRRRRRRRRRRR'

In [84]:
tokenizer.decode([5])

' the'

In [85]:
_out.squeeze(0).tolist()

[0,
 4917,
 12479,
 119,
 12601,
 1071,
 11,
 127,
 30475,
 5048,
 12,
 38210,
 248,
 947,
 387,
 344,
 7042,
 4,
 653,
 197,
 38,
 1606,
 7,
 116,
 2,
 50118,
 4783,
 4783,
 863,
 510,
 510,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500]

In [15]:
scores

defaultdict(list, {'AddToPlaylist': []})

In [68]:
response

defaultdict(list,
            {'AddToPlaylist': [['Add',
               'another',
               'song',
               'to',
               'the',
               'Cita',
               'RomÃ¡ntica',
               'playlist.',
               'Where',
               'should',
               'I',
               'add?']],
             'BookRestaurant': [['<s>book',
               'The',
               'Middle',
               'East',
               'restaurant',
               'in',
               'IN',
               'for',
               'noon',
               'Where',
               'do',
               'they',
               'want',
               'to',
               'eat?']],
             'GetWeather': [['<s>What',
               'will',
               'the',
               'weather',
               'be',
               'this',
               'year',
               'in',
               'Horseshoe',
               'Lake',
               'State',
               'Fish',
             